In [10]:
import pandas as pd

import bs4
from selenium import webdriver
from selenium.webdriver.common.by import By

In [12]:
def date_to_ints(date: str):
    d = {'mon': '1', 'tue' : '2', 'wed': '3', 'thu' : '4', 'fri': '5', 'sat': '6', 'sun': '7'}
    m = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06', 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12', }
    d1 = date[1].string.split(',')
    d2 = d1[1].strip().lower().split('-')
    d2[1] = m[d2[1]]
    date =  '-'.join(d2) + '-' + d[d1[0].lower()] + '-' + '-'.join(date[0].string.split(':'))
    return date

In [13]:
BASE_URL = 'https://www.whoscored.com/'
SEASON_URL = "Regions/252/Tournaments/2/Seasons/8228/England-Premier-League"
GAME_LINKS = []

driver = webdriver.Chrome()
driver.get(BASE_URL+SEASON_URL)
source = bs4.BeautifulSoup(driver.page_source)

SEASON = source.find('option', attrs={'value':f"/{SEASON_URL}", "selected": "selected"}).string.replace('/','-')

In [14]:
def append_links():
    game_tables = source.find_all('a', attrs={'class': 'match-report'})
    for game_table in game_tables:
        game_link = game_table['href']
        game_link = game_link.split('/')
        game_link[3] = 'PlayerStatistics'
        GAME_LINKS.append('/'.join(game_link))

In [15]:
append_links()  
while source.find('a', attrs={'class': ['previous']})['title'][0] != 'N':
    driver.find_element(By.CLASS_NAME, 'previous').click()
    source = bs4.BeautifulSoup(driver.page_source)
    append_links()

driver.close()
print(len(GAME_LINKS))

416


In [16]:
GAME_LINKS = list(set(GAME_LINKS))
print(GAME_LINKS[0])

/Matches/1485428/PlayerStatistics/England-Premier-League-2020-2021-Arsenal-Manchester-United


In [17]:
GAMES_DATA = []

for game_link in GAME_LINKS:
    driver = webdriver.Chrome()
    driver.get(BASE_URL+game_link)

    game_data = {}
    game_source = bs4.BeautifulSoup(driver.page_source)

    players = game_source.find_all('a', attrs={'class': 'player-link'})
    player_names = []
    for i in range(0, len(players), 2):
        player_names.append(' '.join(players[i]['href'].split('/')[-1].split('-')))
    game_data['name'] = player_names

    teams = game_source.find_all('span', attrs={'class': 'team-name'})
    team_names = []
    for i in range(0, len(teams), 2):
        team_names.append(teams[i].string.split(',')[0])
    game_data['team'] = team_names

    numbers_and_positions = game_source.find_all('span', attrs={'class': 'player-meta-data'})
    team_numbers = []
    team_positions = []
    for i in range(0, len(numbers_and_positions), 4):
        team_numbers.append(numbers_and_positions[i].string)
        team_positions.append(','.join([position.strip() for position in numbers_and_positions[i+1].string.split(',') if position != '']))
    game_data['number'] = team_numbers
    game_data['position'] = team_positions

    minsPlayed = game_source.find_all('td', attrs={'class': 'minsPlayed'})
    mins_played = []
    for i in range(0, len(minsPlayed), 1):
        mins_played.append(0 if minsPlayed[i].string.strip() == '-' else int(minsPlayed[i].string))
    game_data['min'] = mins_played

    goals = game_source.find_all('td', attrs={'class': 'goal'})
    goals_scored = []
    for i in range(0, len(goals), 1):
        goals_scored.append(0 if goals[i].string.strip() == '-' else int(goals[i].string))
    game_data['goal'] = goals_scored

    assits = game_source.find_all('td', attrs={'class': 'assistTotal'})
    goals_assisted = []
    for i in range(0, len(assits), 1):
        goals_assisted.append(0 if assits[i].string.strip() == '-' else int(assits[i].string))
    game_data['assist'] = goals_assisted

    yellows = game_source.find_all('td', attrs={'class': 'yellowCard'})
    yellow_cards = []
    for i in range(0, len(yellows), 1):
        yellow_cards.append(0 if yellows[i].string.strip() == '-' else int(yellows[i].string))
    game_data['yellow'] = yellow_cards

    reds = game_source.find_all('td', attrs={'class': 'redCard'})
    red_cards = []
    for i in range(0, len(reds), 1):
        red_cards.append(0 if reds[i].string.strip() == '-' else int(reds[i].string))
    game_data['red'] = red_cards

    shots = game_source.find_all('td', attrs={'class': 'shotsPerGame'})
    shots_per_game = []
    for i in range(0, len(shots), 1):
        shots_per_game.append(0 if shots[i].string.strip() == '-' else float(shots[i].string))
    game_data['shot'] = shots_per_game

    passes = game_source.find_all('td', attrs={'class': 'passSuccess'})
    pass_success = []
    for i in range(0, len(passes), 1):
        pass_success.append(0 if passes[i].string.strip() == '-' else float(passes[i].string)/100)
    game_data['pass'] = pass_success

    aers = game_source.find_all('td', attrs={'class': 'aerialWonPerGame'})
    aer_wins = []
    for i in range(0, len(aers), 1):
        aer_wins.append(0 if aers[i].string.strip() == '-' else float(aers[i].string))
    game_data['aer'] = aer_wins
    
    moms = game_source.find_all('td', attrs={'class': 'manOfTheMatch'})
    mom = []
    for i in range(0, len(moms), 1):
        mom.append(0 if moms[i].string.strip() == '-' else int(moms[i].string))
    game_data['mom'] = mom

    ratings = game_source.find_all('td', attrs={'class': 'rating'})
    game_rating = []
    for i in range(0, len(ratings), 1):
        game_rating.append(0 if ratings[i].string.strip() == '-' else float(ratings[i].string))
    game_data['rating'] = game_rating

    # driver = webdriver.Chrome()By.id
    game_link = game_link.split('/')
    # game_link[3] = 'Show'
    # driver.get(BASE_URL+'/'.join(game_link))
    DATE = date_to_ints(game_source.find_all('div', attrs={'class': 'info-block'})[-1].find_all('dd'))

    # driver.close()

    GAMES_DATA.append(pd.DataFrame(game_data))

    pd.DataFrame(game_data).to_csv(path_or_buf = f"./seasons/{SEASON_URL.split('/')[-1]}/{SEASON}/{DATE+'-'+game_link[2]}.csv", index=False)

In [9]:
driver = webdriver.Chrome()
driver.get("https://www.whoscored.com//Matches/1485428/PlayerStatistics/England-Premier-League-2020-2021-Arsenal-Manchester-United")

source = bs4.BeautifulSoup(driver.page_source)
print(source.find_all('div', attrs={'class': 'info-block'})[-1].find_all('dd'))

driver.close()

[<dd>17:30</dd>, <dd>Sat, 30-Jan-21</dd>]
